# BasicTick: Delete Everything


In [1]:
import os
import subprocess
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env import *

# Cluster names and database
from basictick_setup import *

# Selective Deletion or All
DELETE_ALL=True
#DELETE_ALL=False

DELETE_CLUSTERS=True
DELETE_SCALING_GROUP=False
DELETE_DATAVIEW=False
DELETE_VOLUME=False
DELETE_DATABASE=False

if DELETE_DATABASE:
    DELETE_DATAVIEW=True

if DELETE_VOLUME:
    DELETE_DATAVIEW=True

if DELETE_ALL:
    DELETE_CLUSTERS=True
    DELETE_SCALING_GROUP=True
    DELETE_DATAVIEW=True
    DELETE_VOLUME=True
    DELETE_DATABASE=True


In [2]:
# triggers credential get
session=None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using Defaults ...


# Delete All Resources

Using service APIs to delete all resources, if resource does not exist, assume it was already deleted and continue.

## APIs Used
[get_kx_cluster](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_cluster.html)  
[delete_kx_cluster](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/delete_kx_cluster.html)  
[get_kx_dataview](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_dataview.html)  
[delete_kx_dataview](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/delete_kx_dataview.html)  
[get_kx_volume](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_volume.html)  
[delete_kx_volume](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/delete_kx_volume.html)  
[get_kx_scaling_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_scaling_group.html)  
[delete_kx_scaling_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_scaling_group.html)  
[get_kx_database](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_database.html)  
[delete_kx_database](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/delete_kx_database.html)  

## Kill FeedHandler and Others
You can connect to the running (not finspace cluster) q instance and have it exit.

### Example
```
$ q
q) h:hopen`::5030
q) @[h; "exit 0", {}]
```

Here we do the equivalent from Python

In [3]:
print("Stopping local FeedHandler ...")
try:
    with kx.QConnection(port=FH_PORT) as q:
        q("exit 0") 
except RuntimeError:
    print("Stopped")
except:
    print("Not Running")
    
print("Stopping local Subscriber ...")
try:
    with kx.QConnection(port=SUBSCRIBER_PORT) as q:
        q("exit 0") 
except RuntimeError:
    print("Stopped")
except:
    print("Not Running")    

Stopping local FeedHandler ...
Stopped
Stopping local Subscriber ...
Stopped


In [4]:
# Clusters
#-------------------------------------------------------------------------------------------
delete_status = ['RUNNING', 'CREATING', 'ACTIVE', 'CREATE_FAILED']
deleting_status = ['DELETING']

if DELETE_CLUSTERS:
    print("Deleting Clusters...")

    for k,v in all_clusters.items():
        print(f"\tDelete: {v}")
        try:
            client.delete_kx_cluster(environmentId=ENV_ID, clusterName=v)
            print(f"\tDeleted: {v}")
        except:
            print(f"\tAlready Deleted? {v}")
            next

    print(f"** Deleted {len(all_clusters)} Clusters **")

# Dataview
#-------------------------------------------------------------------------------------------
if DELETE_DATAVIEW:
    print()
    print(f"Deleting Dataview: {DBVIEW_NAME}...")
    this_view = get_kx_dataview(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)

    if this_view is None:
        print(f"Not Found: {DBVIEW_NAME}")
    elif this_view['status'] in deleting_status:
        wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, status='DELETED', show_wait=True)
        print("** Deleted View **")
    elif this_view['status'] not in delete_status:
        print(f"Cannot Delete: {DBVIEW_NAME} Status: {this_view['status']}")
    else:
        print(f"delete: {DBVIEW_NAME}")
        client.delete_kx_dataview(environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)
        wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, status='DELETED', show_wait=True)
        print("** Deleted View **")

# Volume
#-------------------------------------------------------------------------------------------
if DELETE_VOLUME:
    print()
    print(f"Deleting Volume: {VOLUME_NAME}...")
    this_volume = get_kx_volume(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME)

    if this_volume is None:
        print(f"Not Found: {VOLUME_NAME}")
    elif this_volume['status'] in deleting_status:
        wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, status='DELETED', show_wait=True)
        print("** Deleted Volume **")
    elif this_volume['status'] not in delete_status:
        print(f"Cannot Delete: {VOLUME_NAME} Status: {this_volume['status']}")
    else:
        print(f"Delete: {VOLUME_NAME}")
        client.delete_kx_volume(environmentId=ENV_ID, volumeName=VOLUME_NAME)
        wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, status='DELETED', show_wait=True)
        print("** Deleted Volume **")

# Volume
#-------------------------------------------------------------------------------------------
if DELETE_DATABASE:
    print()
    print(f"Deleting Database: {DB_NAME}...")
    this_database = get_kx_database(client=client, environmentId=ENV_ID, databaseName=DB_NAME)

    if this_database is None:
        print(f"Not Found: {DB_NAME}")
    else:
        print(f"Delete: {VOLUME_NAME}")
        client.delete_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)
        print("** Deleted Database **")
        
        
# Scaling Group
#-------------------------------------------------------------------------------------------
if DELETE_SCALING_GROUP:
    print()
    print(f"Deleting Scaling Group: {SCALING_GROUP_NAME}...")
    this_sg = get_kx_scaling_group(client=client, environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME)

    if this_sg is None:
        print(f"Not Found: {SCALING_GROUP_NAME}")
    elif this_sg['status'] in deleting_status:
        wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME, status='DELETED', show_wait=True)
        print("** Deleted Scaling Group **")
    elif this_sg['status'] not in delete_status:
        print(f"Cannot Delete: {SCALING_GROUP_NAME} Status: {this_sg['status']}")
    else:
        print(f"Delete: {SCALING_GROUP_NAME}")
        client.delete_kx_scaling_group(environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME)
        wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME, status='DELETED', show_wait=True)
        print("** Deleted Scaling Group **")

print("\nDone Deleting")

Deleting Clusters...
	Delete: TP_basictickdb
	Deleted: TP_basictickdb
	Delete: RDB_basictickdb
	Deleted: RDB_basictickdb
	Delete: HDB_basictickdb
	Deleted: HDB_basictickdb
	Delete: GATEWAY_basictickdb
	Deleted: GATEWAY_basictickdb
	Delete: CALC_basictickdb
	Deleted: CALC_basictickdb
** Deleted 5 Clusters **

Deleting Dataview: basictickdb_DBVIEW...
delete: basictickdb_DBVIEW
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:00:00, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:00:30, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:01:00, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:01:30, waiting 30 sec ...
Dataview: basictickdb_DBVIEW not found
** Deleted View **

Deleting Volume: RDB_TP_SHARED...
Delete: RDB_TP_SHARED
Volume: RDB_TP_SHARED not found
** Deleted Volume **

Deleting Database: basictickdb...
Delete: RDB_TP_SHARED
** Deleted Database **

Deleting Scaling Grou

In [5]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-08-08 18:39:00.613741
